# Лабораторная работа № 3
## Двойственный симплекс метод
## Дополнительное задание

### Выполнил: Яковлев Артур, 853501
### Проверила: Костюкова О.И.

### Вариант 7

Импорт необходиых модулей

In [2]:
import numpy as np
import scipy.linalg as sla
from copy import deepcopy

Исходные данные для пункта а)

In [137]:
dataset = {
    'A': np.array([
        [ 1., -1., 0.,  2., -1., 1., 2., 1., 7.],
        [-1.,  2., 1.,  3.,  0., 1., 1., 1., 1.],
        [ 1., -1., 1.,  0.,  1., 4., 4., 1., 2.],
        [ 1., -3., 0., -8.,  2., 3., 0., 1., 1.]
    ]),
    'c': np.array([12., -24., -6., -34., -1., 4., 7., 3., 23.]),
    'b': np.array([2.5, 22., 36.5, -18.]),
    'lower': np.array([-1., -4., -1., 4., -3., 4., 0., -1., -4.]),
    'upper': np.array([3., 0., 6., 7., 4., 8., 4., 4., 1.]),
    'Jb': [0, 2, 5, 8],
}
eps = 1e-9

Исходные данные для пункта б)

In [138]:
dataset_b = {
    'A': np.array([
        [ 1., -1., 0.,  2., -1., 1., 2., 1., 7.],
        [-1.,  2., 1.,  3.,  0., 1., 1., 1., 1.],
        [ 1., -1., 1.,  0.,  1., 4., 4., 1., 2.],
        [ 1., -3., 0., -8.,  2., 3., 0., 1., 1.]
    ]),
    'c': np.array([12., -24., -6., -34., -1., 4., 7., 3., 23.]),
    'b': np.array([2.5, 22., 36.5, -18.]),
    'lower': np.array([-1., -4., -1., 4., -3., 4., 0., -1., -4.]),
    'upper': np.array([3., 0., 6., 7., 4., 8., 4., 4., 1.]),
    'Jb': [0, 2, 3, 4],
}

Исходные данные для пункта в)

In [139]:
dataset_c = {
    'A': np.array([
        [ 1., -1., 0.,  2., -1., 1., 2., 1., 7.],
        [-1.,  2., 1.,  3.,  0., 1., 1., 1., 1.],
        [ 1., -1., 1.,  0.,  1., 4., 4., 1., 2.],
        [ 1., -3., 0., -8.,  2., 3., 0., 1., 1.]
    ]),
    'c': np.array([12., -24., -6., -34., -1., 4., 7., 3., 23.]),
    'b': np.array([2., -20., -4., -10.]),
    'lower': np.array([-1., -4., -1., 4., -3., 4., 0., -1., -4.]),
    'upper': np.array([3., 0., 6., 7., 4., 8., 4., 4., 1.]),
    'Jb': [0, 2, 5, 8],
}

Исходные данные для пункта г)

In [140]:
dataset_d = {
    'A': np.array([
        [ 1., -1., 0.,  2., -1., 1., 2., 1., 7.],
        [-1.,  2., 1.,  3.,  0., 1., 1., 1., 1.],
        [ 1., -1., 1.,  0.,  1., 4., 4., 1., 2.],
        [ 1., -3., 0., -8.,  2., 3., 0., 1., 1.]
    ]),
    'c': np.array([12., -24., -6., -34., -1., 4., 7., 3., 23.]),
    'b': np.array([2.5, 22., 36.5, -18.]),
    'lower': np.array([-1., -4., -1., 4., -3., 4., 0., -1., -4.]),
    'upper': np.array([3., 0., -1., 7., -3., 8., 4., 4., 1.]),
    'Jb': [0, 2, 5, 8],
}

Реализация двойственного симплекс метода с ограничениями

In [147]:
def solve(data, max_iters=20, verbose=True):
    dataset = deepcopy(data)
    B = sla.inv(dataset['A'][:, dataset['Jb']])
    
    if verbose:
        print(f'A_b = {dataset["A"][:, dataset["Jb"]]}')
        print(f'c_b = {dataset["c"][dataset["Jb"]]}')
        print(f'B = {B}')
        print(f'Jb = {dataset["Jb"]}')
    
    # Step 1
    y = dataset['c'][dataset['Jb']] @ B
    delta_j = y @ dataset['A'] - dataset['c']
    Jn = [i for i in range(dataset['c'].shape[0]) if i not in dataset['Jb']]
    Jn_pos = [i for i in Jn if delta_j[i] > -eps]
    Jn_neg = [i for i in Jn if delta_j[i] <= -eps]
    
    if verbose:
        print('\nStep 1\n')
        print(f'y = {y}')
        print(f'delta j = {delta_j}')
        print(f'Jn = {Jn}')
        print(f'Jn+ = {Jn_pos}')
        print(f'Jn- = {Jn_neg}')
    
    for it in range(max_iters):
        # Step 2
        x = np.zeros(dataset['c'].shape[0])
        x[Jn_pos] = dataset['lower'][Jn_pos]
        x[Jn_neg] = dataset['upper'][Jn_neg]
        x[dataset['Jb']] = B.dot(dataset['b'] - np.sum(dataset['A'][:, Jn] * x[Jn], axis=1))
        # x[dataset['Jb']] = B.dot(dataset['b'] - sum([dataset['A'][:, j].dot(x[j]) for j in Jn]))
        
        if verbose:
            print('\n===========================================\n')
            print(f'Iteration {it + 1}')
            print('\nStep 2\n')
            print(f'x = {x}')

        # Step 3
        if np.all(dataset['lower'] < x + eps) and np.all(x - eps < dataset['upper']):
            print(f'\nOptimal plan: {x}\n')
            print(f'cx = {dataset["c"] @ x}')
            return

        # Step 4
        for i in range(len(dataset['Jb'])):
            j_i = dataset['Jb'][i]
            if dataset['lower'][j_i] > x[j_i] or x[j_i] > dataset['upper'][j_i]:
                j_k = i
                break
        
        if verbose:
            print('\nStep 4\n')
            print(f'jk = {dataset["Jb"][j_k]}')

        # Step 5
        mu_jk = 1 if x[dataset["Jb"][j_k]] < dataset['lower'][dataset["Jb"][j_k]] else -1
        delta_y = mu_jk * B[j_k]
        mu_j = delta_y @ dataset['A']
        
        if verbose:
            print('\nStep 5\n')
            print(f'mu_jk = {mu_jk}')
            print(f'delta y = {delta_y}')
            print(f'mu_j = {mu_j}')

        # Step 6
        sigma_j = np.array([np.inf] * dataset['c'].shape[0])
        for i in Jn_pos:
            if mu_j[i] < eps:
                sigma_j[i] = -delta_j[i] / mu_j[i]

        for i in Jn_neg:
            if mu_j[i] > -eps:
                sigma_j[i] = -delta_j[i] / mu_j[i]
        
        j0 = np.argmin(sigma_j)
        sigma0 = sigma_j[j0]
        # To prevent -inf as sigma0
        while sigma0 == -np.inf:
            sigma_j[j0] = np.inf
            j0 = np.argmin(sigma_j)
            sigma0 = sigma_j[j0]
        
        if verbose:
            print('\nStep 6\n')
            print(f'sigma_j = {sigma_j}')
            print(f'j* = {j0}')
            print(f'sigma0 = {sigma0}')

        # Step 7
        if sigma0 == np.inf:
            print('\nSolution does not exist')
            return

        # Step 8
        delta_j += sigma0 * mu_j
        dataset['Jb'].pop(j_k)
        delta_j[dataset['Jb']] = 0.
        dataset['Jb'] = [j0] + dataset['Jb']
        # dataset['Jb'].sort()
        
        if verbose:
            print('\nStep 8\n')
            print(f'New delta j = {delta_j}')
            print(f'Jb = {dataset["Jb"]}')
        
        # Step 9
        B = sla.inv(dataset['A'][:, dataset['Jb']])
        Jn = [i for i in range(dataset['c'].shape[0]) if i not in dataset['Jb']]
        Jn_pos = [i for i in Jn if delta_j[i] > -eps]
        Jn_neg = [i for i in Jn if delta_j[i] <= -eps]
        
        if verbose:
            print('\nStep 9\n')
            print(f'A_b = {dataset["A"][:, dataset["Jb"]]}')
            print(f'B = {B}')
            print(f'Jn = {Jn}')
            print(f'Jn+ = {Jn_pos}')
            print(f'Jn- = {Jn_neg}')
        
    print(f'\nBest found solution: {x}')

**Задача а)**

In [148]:
solve(dataset)

A_b = [[ 1.  0.  1.  7.]
 [-1.  1.  1.  1.]
 [ 1.  1.  4.  2.]
 [ 1.  0.  3.  1.]]
c_b = [12. -6.  4. 23.]
B = [[ 0.   -1.    1.   -1.  ]
 [-0.1  -0.4   1.4  -1.7 ]
 [-0.05  0.3  -0.3   0.65]
 [ 0.15  0.1  -0.1   0.05]]
Jb = [0, 2, 5, 8]

Step 1

y = [ 3.85 -6.1   0.1   1.95]
delta j = [ 1.77635684e-15  2.00000000e+00  0.00000000e+00  7.80000000e+00
  1.15000000e+00  3.55271368e-15 -5.00000000e+00 -3.20000000e+00
  7.10542736e-15]
Jn = [1, 3, 4, 6, 7]
Jn+ = [1, 3, 4]
Jn- = [6, 7]


Iteration 1

Step 2

x = [ 1.5   -4.    13.35   4.    -3.     2.175  4.     4.    -4.025]

Step 4

jk = 2

Step 5

mu_jk = -1
delta y = [ 0.1  0.4 -1.4  1.7]
mu_j = [ 0.00000000e+00 -3.00000000e+00 -1.00000000e+00 -1.22000000e+01
  1.90000000e+00  8.88178420e-16 -5.00000000e+00  8.00000000e-01
  6.66133815e-16]

Step 6

sigma_j = [       inf 0.66666667        inf 0.63934426        inf        inf
        inf 4.                inf]
j* = 3
sigma0 = 0.6393442622950818

Step 8

New delta j = [ 0.          0.08196

/home/pineapple/.local/lib/python3.6/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in double_scalars


**Задача б)**

In [149]:
solve(dataset_b, verbose=False)


Optimal plan: [ 1.375      -1.58333333  6.          4.         -3.          4.66666667
  4.          3.875      -4.        ]

cx = -148.20833333333331


/home/pineapple/.local/lib/python3.6/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in double_scalars


**Задача в)**

In [152]:
solve(dataset_c, verbose=False)


Solution does not exist


/home/pineapple/.local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in double_scalars


**Задача г)**

In [153]:
solve(dataset_d, verbose=False)


Optimal plan: [ 0.33653846  0.         -1.          4.17307692 -3.          7.11538462
  4.          3.70192308 -4.        ]

cx = -153.2788461538462


/home/pineapple/.local/lib/python3.6/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in double_scalars
